## Imports

In [ ]:
import getpass
import logging
from pathlib import Path
import sys
from typing import Optional, NoReturn
import datetime
import zoneinfo
import time

import chromedriver_binary
import undetected_chromedriver
from selenium import webdriver
from selenium.webdriver.remote.webelement import WebElement
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions
from selenium.webdriver.common.action_chains import ActionChains

## Logging

In [ ]:
file_handler = logging.FileHandler('shop-Main.log', mode='w')
file_handler.setLevel(logging.DEBUG)

console_handler = logging.StreamHandler(stream=sys.stdout)
console_handler.setLevel(logging.INFO)

logging.basicConfig(
    level=logging.DEBUG,
    format='%(asctime)s.%(msecs)03d %(levelname)s %(module)s - %(funcName)s: %(message)s',
    datefmt='%Y-%m-%d %H:%M:%S',
    handlers=(file_handler, console_handler),
)

## Constants

In [ ]:
TIMEOUT = 100

URL_LOGIN = 'https://www.tesco.com/account/login/en-GB?from=/'
EMAIL_LOGIN = 'roseannaferguson@hotmail.com'
PASSWORD_LOGIN = getpass.getpass()

## Functions

In [ ]:
def wait_and_get(
    driver: webdriver.Chrome,
    value: str,
    by: Optional[By] = None,
    timeout: Optional[int] = None,
) -> WebElement:
    
    if by is None:
        by = By.XPATH
        
    if timeout is None:
        timeout = TIMEOUT
        
    logging.debug(f'{by=}, {value=}, {timeout=}')
    
    wait = expected_conditions.presence_of_element_located((by, value))
    WebDriverWait(driver, timeout).until(wait)
    
    logging.info(f'Waited for: {by=}, {value=}')
    time.sleep(1)

    return driver.find_element(by=by, value=value)
    

def wait_and_click(
    driver: webdriver.Chrome,
    value: str,
    by: Optional[By] = None,
    timeout: Optional[int] = None,
) -> NoReturn:
    element = wait_and_get(driver=driver, value=value, by=by, timeout=timeout)
    element.click()
    logging.info(f'Clicked: {by=}, {value=}')

## Get Driver

In [ ]:
options = undetected_chromedriver.ChromeOptions()
options.add_argument('--start-maximized')
options.add_argument('--password-store=basic')
options.add_experimental_option(
    'prefs',
    {
        'credentials_enable_service': False,
        'profile.password_manager_enabled': False,
    },
)

driver = undetected_chromedriver.Chrome(options=options)

## Go to URL

In [ ]:
driver.get(URL_LOGIN)

## Login

In [ ]:
xpath_cookies_accept = '//button[@type="submit"]//span[text()="Accept all cookies"]'
wait_and_click(driver=driver, value=xpath_cookies_accept)

xpath_email = '//input[@id="email"]'
element_email = wait_and_get(driver=driver, value=xpath_email)
element_email.send_keys(EMAIL_LOGIN)
logging.info(f'Sent keys: {EMAIL_LOGIN=} into {xpath_email=}')

xpath_password = '//input[@id="password"]'
element_password = driver.find_element(by=By.XPATH, value=xpath_password)
element_password.send_keys(PASSWORD_LOGIN)
logging.info(f'Sent keys: password into {xpath_password=}')
time.sleep(1)

xpath_sign_in = '//button[@id="signin-button"]'
element_sign_in = driver.find_element(by=By.XPATH, value=xpath_sign_in)
element_sign_in.click()
logging.info(f'Clicked {xpath_sign_in=}')

## Go to book a slot

In [ ]:
xpath_groceries = '//a[@title="Groceries"]'
wait_and_click(driver=driver, value=xpath_groceries)

xpath_shop_groceries = '//span[text()="Shop groceries"]'
element_span_shop_groceries = wait_and_get(driver=driver, value=xpath_shop_groceries)
element_shop_groceries = element_span_shop_groceries.find_element(by=By.XPATH, value='../..')
element_shop_groceries.click()

xpath_book_another_slot = '//a[text()="Book another slot"]'
wait_and_click(driver=driver, value=xpath_book_another_slot)

xpath_delivery = '//a[@href="/groceries/en-GB/slots/delivery"]'
wait_and_click(driver=driver, value=xpath_delivery)

## Book the slot

In [ ]:
now = datetime.datetime.now(tz=zoneinfo.ZoneInfo('Europe/London'))

start_week_third = now + datetime.timedelta(days=21)
end_week_third = start_week_third + datetime.timedelta(days=6)

xpath_week_third = '//a[@id="between-september 18th - 24th"]'
wait_and_click(driver=driver, value=xpath_week_third)

days_to_saturday = (5 - start_week_third.weekday()) % 7
date_saturday = start_week_third + datetime.timedelta(days=days_to_saturday)

datetime_slot_start = date_saturday.replace(hour=9, minute=0, second=0)
datetime_slot_end = datetime_slot_start + datetime.timedelta(hours=1)

def suffix(day_of_month: int):
    if 11 <= day_of_month <= 13:
        return 'th' 
    else:
        return {1: 'st', 2: 'nd', 3: 'rd'}.get(day_of_month % 10, 'th')
    
date_formatted = datetime_slot_start.strftime(f'%A %d{suffix(datetime_slot_start.day)} %B')
text_slot = f'{date_formatted}, Between {datetime_slot_start:%H:%M} - {datetime_slot_end:%H:%M}.'

xpath_slot_saturday = f'//span[text()="{text_slot}"]'
element_slot_saturday = wait_and_get(driver=driver, value=xpath_slot_saturday)
element_slot_button = element_slot_saturday.find_element(by=By.XPATH, value='..')
element_slot_button.click()

xpath_my_orders = '//span[text()="My orders"]'
wait_and_click(driver=driver, value=xpath_my_orders)

## Add all to basket

In [ ]:
xpath_add_all_to_basket = '//button[@type="submit"]//span[text()="Add all to basket"]'
wait_and_click(driver=driver, value=xpath_add_all_to_basket)

## Checkout

In [ ]:
xpath_span_checkout = '//span[text()="Checkout"]'
element_span_checkout = wait_and_get(driver=driver, value=xpath_span_checkout)
driver.execute_script("arguments[0].click();", element_span_checkout)

time.sleep(1)

xpath_a_checkout = '//a[text()="Checkout"]'
element_a_checkout = wait_and_get(driver=driver, value=xpath_a_checkout)
driver.execute_script("arguments[0].click();", element_a_checkout)

time.sleep(1)

xpath_a_continue_checkout = '//a[text()="Continue checkout"]'
element_a_continue_checkout = wait_and_get(driver=driver, value=xpath_a_continue_checkout)
driver.execute_script("arguments[0].click();", element_a_continue_checkout)

time.sleep(1)

xpath_a_continue_to_payment = '//a[text()="Continue to payment"]'
element_a_continue_to_payment = wait_and_get(driver=driver, value=xpath_a_continue_to_payment)
driver.execute_script("arguments[0].click();", element_a_continue_to_payment)

## Input card details

In [ ]:
# driver.switch_to.frame('bounty-iframe')

# xpath_cvc = '//input[@id="card-cvc"]'
# element_cvc = wait_and_get(driver=driver, value=xpath_cvc)
# element_cvc.send_keys(getpass.getpass())

# xpath_confirm_order = '//input[@value="Confirm order"]'
# wait_and_click(driver=driver, value=xpath_confirm_order)